In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np
np.random.seed(42)

In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_shape=(self.latent_dim,)))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train.astype(np.float32) / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Sample generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        #fig.suptitle("DCGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [4]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

W1008 02:31:49.932593 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1008 02:31:49.934046 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1008 02:31:49.936102 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1008 02:31:49.950605 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

W1008 02:31:51.416709 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1008 02:31:51.426113 140469632849664 deprecation.py:323] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1008 02:31:51.469474 140469632849664 deprecation_wrapper.py:119] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)       0         
__________

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

3371 [D loss: 0.750607, acc.: 53.12%] [G loss: 0.738581]
3372 [D loss: 0.749473, acc.: 43.75%] [G loss: 0.824723]
3373 [D loss: 0.822625, acc.: 46.88%] [G loss: 0.824153]
3374 [D loss: 0.714348, acc.: 50.00%] [G loss: 0.913441]
3375 [D loss: 0.630111, acc.: 65.62%] [G loss: 0.725744]
3376 [D loss: 0.689124, acc.: 56.25%] [G loss: 0.977828]
3377 [D loss: 0.717194, acc.: 46.88%] [G loss: 0.851488]
3378 [D loss: 0.625076, acc.: 62.50%] [G loss: 1.042072]
3379 [D loss: 0.627858, acc.: 68.75%] [G loss: 0.982343]
3380 [D loss: 0.798010, acc.: 43.75%] [G loss: 0.826565]
3381 [D loss: 0.609593, acc.: 59.38%] [G loss: 0.914966]
3382 [D loss: 0.653115, acc.: 59.38%] [G loss: 0.830844]
3383 [D loss: 0.732731, acc.: 56.25%] [G loss: 0.880097]
3384 [D loss: 0.619515, acc.: 68.75%] [G loss: 0.867517]
3385 [D loss: 0.753634, acc.: 43.75%] [G loss: 0.999360]
3386 [D loss: 0.886871, acc.: 37.50%] [G loss: 0.886881]
3387 [D loss: 0.535218, acc.: 78.12%] [G loss: 0.767246]
3388 [D loss: 0.742315, acc.: 5

3516 [D loss: 0.639330, acc.: 62.50%] [G loss: 0.781666]
3517 [D loss: 0.653950, acc.: 62.50%] [G loss: 0.807343]
3518 [D loss: 0.691799, acc.: 56.25%] [G loss: 0.748130]
3519 [D loss: 0.656960, acc.: 62.50%] [G loss: 0.882347]
3520 [D loss: 0.639679, acc.: 68.75%] [G loss: 0.775941]
3521 [D loss: 0.758126, acc.: 46.88%] [G loss: 0.764297]
3522 [D loss: 0.626458, acc.: 65.62%] [G loss: 0.900950]
3523 [D loss: 0.643643, acc.: 62.50%] [G loss: 0.801245]
3524 [D loss: 0.691249, acc.: 46.88%] [G loss: 0.909890]
3525 [D loss: 0.686298, acc.: 62.50%] [G loss: 0.862961]
3526 [D loss: 0.670164, acc.: 59.38%] [G loss: 0.765127]
3527 [D loss: 0.682361, acc.: 50.00%] [G loss: 0.719938]
3528 [D loss: 0.642036, acc.: 62.50%] [G loss: 0.837784]
3529 [D loss: 0.723528, acc.: 43.75%] [G loss: 0.960359]
3530 [D loss: 0.772733, acc.: 46.88%] [G loss: 0.789777]
3531 [D loss: 0.699592, acc.: 56.25%] [G loss: 0.848504]
3532 [D loss: 0.625403, acc.: 65.62%] [G loss: 0.900694]
3533 [D loss: 0.773310, acc.: 4

3661 [D loss: 0.610797, acc.: 65.62%] [G loss: 0.814973]
3662 [D loss: 0.700645, acc.: 56.25%] [G loss: 1.010085]
3663 [D loss: 0.682366, acc.: 62.50%] [G loss: 1.042388]
3664 [D loss: 0.652820, acc.: 62.50%] [G loss: 0.922665]
3665 [D loss: 0.773731, acc.: 50.00%] [G loss: 0.798924]
3666 [D loss: 0.731146, acc.: 46.88%] [G loss: 0.867305]
3667 [D loss: 0.751576, acc.: 43.75%] [G loss: 0.919810]
3668 [D loss: 0.569957, acc.: 75.00%] [G loss: 0.961352]
3669 [D loss: 0.710320, acc.: 50.00%] [G loss: 0.917532]
3670 [D loss: 0.648354, acc.: 65.62%] [G loss: 0.843384]
3671 [D loss: 0.769469, acc.: 43.75%] [G loss: 0.871325]
3672 [D loss: 0.786233, acc.: 50.00%] [G loss: 0.776128]
3673 [D loss: 0.725987, acc.: 53.12%] [G loss: 0.818017]
3674 [D loss: 0.725927, acc.: 43.75%] [G loss: 1.005650]
3675 [D loss: 0.635251, acc.: 65.62%] [G loss: 0.816058]
3676 [D loss: 0.761954, acc.: 53.12%] [G loss: 0.917938]
3677 [D loss: 0.711793, acc.: 56.25%] [G loss: 0.860219]
3678 [D loss: 0.667460, acc.: 6

3806 [D loss: 0.790649, acc.: 40.62%] [G loss: 0.675451]
3807 [D loss: 0.693274, acc.: 53.12%] [G loss: 0.843197]
3808 [D loss: 0.660490, acc.: 59.38%] [G loss: 0.856647]
3809 [D loss: 0.661476, acc.: 68.75%] [G loss: 0.915490]
3810 [D loss: 0.652572, acc.: 68.75%] [G loss: 0.871851]
3811 [D loss: 0.731639, acc.: 46.88%] [G loss: 0.868749]
3812 [D loss: 0.702680, acc.: 56.25%] [G loss: 0.844038]
3813 [D loss: 0.801589, acc.: 43.75%] [G loss: 0.824693]
3814 [D loss: 0.672096, acc.: 59.38%] [G loss: 0.889014]
3815 [D loss: 0.743627, acc.: 46.88%] [G loss: 0.830211]
3816 [D loss: 0.757333, acc.: 50.00%] [G loss: 0.860801]
3817 [D loss: 0.720101, acc.: 46.88%] [G loss: 0.748286]
3818 [D loss: 0.730891, acc.: 43.75%] [G loss: 0.820656]
3819 [D loss: 0.689215, acc.: 53.12%] [G loss: 0.855263]
3820 [D loss: 0.713068, acc.: 59.38%] [G loss: 0.824765]
3821 [D loss: 0.669474, acc.: 62.50%] [G loss: 0.777728]
3822 [D loss: 0.691276, acc.: 53.12%] [G loss: 0.937304]
3823 [D loss: 0.617987, acc.: 6

3951 [D loss: 0.653255, acc.: 62.50%] [G loss: 0.988125]
3952 [D loss: 0.616396, acc.: 59.38%] [G loss: 0.896916]
3953 [D loss: 0.707215, acc.: 50.00%] [G loss: 0.892114]
3954 [D loss: 0.681782, acc.: 53.12%] [G loss: 0.914790]
3955 [D loss: 0.668464, acc.: 62.50%] [G loss: 1.007836]
3956 [D loss: 0.641974, acc.: 62.50%] [G loss: 0.825466]
3957 [D loss: 0.625602, acc.: 62.50%] [G loss: 0.896094]
3958 [D loss: 0.794643, acc.: 37.50%] [G loss: 0.920387]
3959 [D loss: 0.730554, acc.: 46.88%] [G loss: 0.905702]
3960 [D loss: 0.717059, acc.: 62.50%] [G loss: 0.855971]
3961 [D loss: 0.726014, acc.: 53.12%] [G loss: 0.800793]
3962 [D loss: 0.675108, acc.: 59.38%] [G loss: 0.868767]
3963 [D loss: 0.676054, acc.: 56.25%] [G loss: 0.781299]
3964 [D loss: 0.637278, acc.: 56.25%] [G loss: 0.787883]
3965 [D loss: 0.626165, acc.: 56.25%] [G loss: 1.026004]
3966 [D loss: 0.631634, acc.: 62.50%] [G loss: 0.919022]
3967 [D loss: 0.659814, acc.: 53.12%] [G loss: 0.964505]
3968 [D loss: 0.627954, acc.: 6

In [1]:
from IPython.display import display, Image, HTML
def Display_Images(images, header=None, width="100%"): # to match Image syntax
    if type(width)==type(1): width = "{}px".format(width)
    html = ["<table style='width:{}'><tr>".format(width)]
    if header is not None:
        html += ["<th>{}</th>".format(h) for h in header] + ["</tr><tr>"]

    cols=1
    for image in images:
        print(image)
        html.append("<td><img src='{}' /></td>".format(image))
        cols+=1
        if (cols>3):
            html.append("</tr><tr>")
            cols=1
    html.append("</tr></table>")
    display(HTML(''.join(html)))

In [2]:
import glob
file_map=[]
for image_path in sorted(glob.glob('./images/*.png'), key=os.path.getmtime):
    file_map.append(image_path)

In [3]:
len(file_map)

0

In [4]:
Display_Images([file_map[i] for i in range(len(file_map))],width="100%")